In [16]:
def compress(data):
    result = bytearray()
    i = 0
    data_len = len(data)
    
    while i < data_len:
        best_len, best_dist = 0, 0
        
        for j in range(max(0, i - 255), i):
            k = 0
            while i + k < data_len and data[j + k] == data[i + k]:
                k += 1
            
            if k > best_len:
                best_len, best_dist = k, i - j
        
        if best_len > 2:
            result.extend([128 + best_len, best_dist])
            i += best_len
        else:
            result.append(data[i])
            i += 1
    
    return result

def hex_to_bytes(hex_string):
    return bytes.fromhex(hex_string.replace('\n', ''))

def bytes_to_lua_string(data):
    return ''.join(f'\\{x}' for x in data)

def bytes_to_hex_string(data):
    hex_string = ''.join(f'{x:02x}' for x in data)
    lines = []
    for i in range(0, len(hex_string), 256):
        line = hex_string[i:i+256]
        if len(line) < 256:
            line = line.ljust(256, '0')
        lines.append(line)
    return '\n'.join(lines)

# Example usage
map_data = """
74b6060606060606060606060606c62636b606060606060606b6f6a7a7f6a7a7a7f6a7a740b70694740640c1c1a7f6a7a7a7a7a7a7a7a7f603e503e6e5f60696
70707070707070177070707070b4050515701770656765450505050505053706f6a7a7f6b60606060606060606c6f6a7a7d6a7f6a7a7a7a7f6a7a7a7a7f60695
b50404046466440404040404040404c5b5040404646644043606f6a7a7a7f6f6f6a7a7a740f606947406f6a7a7a7a7a7a7a7a7a7a7a7a7a7a7b7a7a7a7b006a5
7017707070707070707070707075c606062565350607060606060606060606c6f6a7a7f6f6f6f6f6f6f6f6f6f6f6f6a7a7d6f6d6a7a7a7d4e4f6a7a7a7f606a6
b405050505050515671505050505050505050505050505053706f6a7a7a7a7a7a7a7a7a7f64006947406f6a7a7a7a7a7a7a7a7a7a7a7d4e6e603f6a7b0b20694
7070707070a27070a2707070707034f6f6f6f6f6f6f6f640f6f640f6f64003c0c0b2c0a7a7a7a7a7a7a7a7a7a7a7a7a7a7d6f6d5e6e2f2e5d5f6e6e6e6e60696
74c60606060606060606060606060606060606060606060606c6f6a7a7a7a7a7a7a7a7a7a7f606947406f6a7a7a7a7a7a7a7a7a7a7a7d6a7a7f6a7b0b2b20694
7070707070705161707070c2d0d0d0f6a7a7a7a7a7a740a7a7a7a740a74003b1b2b1c1a7a7a7a7a7b0c0a7a7a7a7a7a7d4d5f6a7a7e3f3a7a7f6a7a7a7400696
7406f6b1b2b2c1f6d6f6f6f6f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6a7a7a7a74040a7a7f6f606947406f6a7a7a7a7a7a7a7a7a7a7a7d6a7a7a7a7b140400695
7070707070705262707070d1701700f6a7a7a7a7a7a7a7a7a740a7a7a7a74003c1a7a7f6f6f6f6b0b2b2c0f6f6f6d4e6e5a7f6a7a7d6a7a7a7f6a7a7a7400694
7406f6b1b2b2c1a7d6a7a7a7a7a7a7d6a7a7a7a7f6f6f6f6f6f6f6f6f6f6f6f64040f6f6f6f606947406f6f6f6f6f6f6404040f6f6f6d6f6f6f6b0b2404006a5
7070707070a27070a2d0d0d3707035f6f6f6f6f6f6f6f6f640f6f64040f6f603f6a787f6c60606060606060606b6e5a7a7a7a7f6e6e5a7a7f6a7a7a7a7f60695
8406f6a7b1c1f6f6d6f6f6f6f6f6d4e5f6f6a7a7a7f6c6060606060706060606060606060606c69475b6b6f6f7c606060606060606060606060606060606c694
7070707070707070707070707085b606060606060606060606060606060606b6f6a797f606260404040404043606f6a7a7a7d4e5f6f6f6f6a7a7d4e6e6e60696
8416f6a7a7f6f6a7d5e4a7a7a7a7d6a7a7f6f6a7a7f60626046644040404040404040404040404c5176416f6f6062614646454040464664404045466044404c5
70700000000000000000000000b5040404646444542444040404040404043606e797a7f706957070d17070707406f6a7a7a7d5e4a7a7a7a7a7a7d6a7a7f606a5
7406f6a7f6f6a7a7a7d6a7a7a7a7d6a7a7a7f6f6a7f60694b4050555050505551570706567701770008606f6e7069400000000001700007017707070d11770a2
a2000000000000000000000000b405050505050505c47000d170707000007416e7a7a7f624707070d17070177506f6a7a7a7a7d6a7a7a7a7a7a7d6a7a7f60696
7506e6e6e6e6e4a7a7d5f6f6f6f6e5a7a7a7a7f6a7f6069474c6060606060606062535060735567077701640e7062715771770707070707070707070d1707070
b405051567156567654505050537c60606060606b694700077a2707070007516f7a7a7e725707070a2a270707606f6f6a7a7a7d6a7a7a7a7a7a7d6a7f6f60696
7406f6a7f6a7d6a7a7f6a7a7a7a7f6a7a7a7a7f6a7f606948406f6f6f6d6f6e7f7f6f640404006a57046c640f6b60606072567350607567070707000a2a27070
74c6060606060607060606060606c6f640f6b7f606940000a2a2707070001786e78797f7a6701770a2a270708506f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f606a5
7406f6a7f6a7d6a7f6d6a7a7a7a7a7f6a7a7a7f6a7f606947406e7b7a7d6a7a7a7d4e6e6e6e606a67034f6f6f6f6f6f7f6e7f6404040f47070707000a2a27070
7406f6f6f6f6f6f6f6d6f6f6f6f6f6f6a74040f60694000000d1001770707047240000345700707070d1707074b606070606060606060706060606060606c694
7406f6a7f6a7d6a7f6d5e6e2f2a7d4f6e6e6e6e6e6e606948406e6e5a7d6f6f6f6e5b0c0c0f616941785f687a7a7a7a7a7a7a7a797e770707017700070d17070
7506f6a7a7a7a7a7a7d6a7a7a7a7a7a7a7a740f606940000a2a2000000700000000000000000707070d17070b5040404040404040404040404040404040404c5
7406f6a7f6d4d5e6f6a7a7e3f3e6e5f6a7a7a7a740400694750640a7a7f6a7a7a7f6b1b2c1f706a47084408797a7a7a7a79797a7a7a7707070707000c2d37070
8606e6e6e6e4f6f6f6e5b0c0a7d4e6e6e6e6e5f606947000a2a20000007070007070707070177070a2a27070b4051565656715701770656765051567150505c4
7406e6e6e6e5a7a7f6a7a7d6a7a7a7f6a7a7a7f640400694860640a7f6d6a703a7a7f6b1c1f606a670864040f6f6f6f7f6f6e7f6e7f6c77070707000d1707070
8616f6a7a7f6a7a7b0f6b1c0a7d6a7a7b0c0a7f606967070c3d0d2007070707070177070707070c2a2a2707075c606060606062565350607060606060606b694
7506f6a7f6a7a7a7a7f6e6e5a7a7f6a7a7a7a7f6a7f606948616f7a7f6d5039703d4e6e6e6e616967047246466346664340706060606577070707000a2a27070
7606f6a7f6d6c003b1c0f6c1a7d6a7b0b2b2b2f6f470707070a2a2707017707070707070707070d1707070707034f6f7e7f6404040f6e74040f640f6e7f60694
76064040f6a7a7a7d4e5f6f6f6f6a7a7d4e6e6e6e6e606948606f6a7f6a7a70397e5f6a7a7f606967070177070707070706664666464001770707000a2a27070
7606f6a7f6d503a703e6e6e6e6e5b0b2b2b2c1a77070177070a2a2707070707070707070707070d1707070700000e7a7a7a7a7a787a7a7a7a7a7a740b7f60695
86064040f6f6a7a7d5e4a7a7a7a7a7a7d6a7f6f6b0c006948506f6a7a7f6e6e5a7e7a7a7a7f7f470707070707070707070707070700000707070707070d17070
8506f6a7f6a7a703a7a7f6a7a7a7b1c1b1c1a7a77070000000d100000046256535560000001700c3a2a200000000f687a7a7a7a7a7a7a7a7a7a7a7a7d5e60694
76064040a7f6f6a7a7d6a7a7a7a7a7a7d6f6f6b0b2c106948406f6a7d4e5f6f6f687a7a787977070177070701770707070707017700000707070170070d17070
7406f6c0a7f6e6e5a7f6a7a7a7a7a7a7a7a7a7a77070000000c3d2000086e7e7f7a6000000000000a2a200001700f6a7a7a7a7a7a7a7a7a7a7a7a7a7a7a70695
8506f6a7a7a7f6f6f6d6f6f6f6f6f6f6d6f6a7b0c1f606948416c0a7d6a7a7a7a7a7a7a7a7a770707070707070000000000000707070a2707070000070d17070
7506f6b2d4e5f6f6f6a7a7b0b0c0a7f6f6f6f6f6c77070707070d1707076f7b7f6a670007070707070d170700000f6c0a787a7a7a7a7a7a7a7a7a7a7a7a70696
7406f6b7a7a7d4e6e6e5a7b0b0c0a7a7d5e4b0c1b7f606947406b2c0d6a797a7a7a7a7a7a79770707070707070c2a2d0d0d0d2707070d17070a2a27070d17070
7606f6c1d6a7a7a7a7a7b0c1b1b1c0f6a740404006a57070a2a2d3177086f7e7e72570007070a2a270d170707000e7b1c0a7a7b7e4a7a7a7a7a7a740a7f606a5
7406f6f6f6f6d6f6f6f6b0b2b2b2c0f6f6d6b1f6f6f606947506b2c1d6f6e7f6f6f6f7e7f7e7c7707070a2d0d0d370707070c3a2d0d0d3d0d0a2a2d0d0777070
8506f6f6d6f6f6f6f6b0c1f6f6f6f6f64040404006947070a2a2707070472417345770007070a277d0d370707035f7f6e7f7f6f6d6f6e7f6f6f6e7f640f60694
74b60606060606066606060606060606060606060606c69470470606060706060606060706065770707070707070177070707070707017707070707070707070
74b6060606060606060606060606060606060606c69470707070707070707070707070707070d1707070707085b606060606060606060606060606060606c694
b504040404040414174414666664644454244404040404c517703424646466646664146466647070a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2
b50404040404040404646444542444040404040404c5a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2b5040404646444542444040404040404040404c5"""

# Convert hex string to bytes
map_bytes = hex_to_bytes(map_data)

# Compress the data
compressed_data = compress(map_bytes)

# Convert compressed data to hexadecimal string
hex_string = bytes_to_hex_string(compressed_data)

print(f"\nCompressed map data (Hexadecimal):\n{hex_string}")


Compressed map data (Hexadecimal):
74b6068b01c626368810b6f6a7a78303840440b70694740640c1c185118503f603e503e6e5f60696708601178508b4050515830b65676545058501370684468963c68354d6864586050695b504040464664483068503c5881036843aa7f6f6848340f68480897c8684b7839cb006a5877a868775c6060625653506078ad184c6
8b018380f6d683d6d4e485dc06a683b384ab15671586b2880685ba88ed409080d4e6e603f6a7b0b2069485ffa28503838934886c40860303c0c0b2c08eb8d6f6d5e6e2f2e5d5f6e6830106967483b3950284ba8981908083c3847f888070516183ffc2d0d0d087b54086054003b1b2b1c185c8b08787d4d583f9e3f384408327
838006f6b1b2b2c1f6d686ba870786c7854a834bf69480a7a7b1404006958680526283ffd17017008ab5868303837e845a83acc08362d4e6e58479843f868084d88480865287598c844040849985808710841184aeb0b28380a585ffa28303d0d0d37070358a38863b03f6a787f6c6068701b6e5855f838585ffa7f606958483
d8b1c188f9d4e585f686320788370606c69475b6b6f6f789498a1885ff880585b69021b6f6a797f6062604850136836ca7a784648466d4e6e6e606968416838a8375d5e487ff8481843266448635863bc5176416f6834914646454040464841d5